In [106]:
import pandas as pd
import numpy as np
import config
import geolocate_function
import path
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords

In [107]:
df=pd.read_csv(path.hotel_csv_path)


In [108]:
df, hotel_missing_list = geolocate_function.coordinate_fill(df)


In [109]:
df.drop(columns='Unnamed: 0', inplace =True)
df.Hotel_Address = df.Hotel_Address.str.lower()

I want to separate and add another column, country name from the addresse. because at this stage of development I don't see why i can need whole addresse. So i checked and sow that so far there is only United Kingdom that has 2 words in country name.   

In [110]:
df.Hotel_Address = df.Hotel_Address.str.replace('united kingdom','uk')
df['Country'] = df['Hotel_Address'].apply(lambda x: x.split()[-1] if len(x.split()) > 1 else x)


In [59]:
df.Country.unique()

array(['netherlands', 'uk', 'france', 'spain', 'italy', 'austria'],
      dtype=object)

I can see that there is only 6 countrys. Now i want to remove features that in my opinion is not important

In [111]:
df1 = df.drop(columns=['Additional_Number_of_Scoring', 'Review_Date','Reviewer_Nationality','days_since_review', 'lat','lng','Review_Total_Negative_Word_Counts','Review_Total_Positive_Word_Counts','Total_Number_of_Reviews_Reviewer_Has_Given','Reviewer_Score','Total_Number_of_Reviews', 'Negative_Review','Positive_Review'])

In [121]:
df1.head()

,Hotel_Address,Average_Score,Hotel_Name,Tags,Country
0,s gravesandestraat 55 oost 1092 aa amsterdam ...,7.7,Hotel Arena,leisure trip couple duplex double room ...,netherlands
1,s gravesandestraat 55 oost 1092 aa amsterdam ...,7.7,Hotel Arena,leisure trip couple duplex double room ...,netherlands
2,s gravesandestraat 55 oost 1092 aa amsterdam ...,7.7,Hotel Arena,leisure trip family with young children d...,netherlands
3,s gravesandestraat 55 oost 1092 aa amsterdam ...,7.7,Hotel Arena,leisure trip solo traveler duplex double ...,netherlands
4,s gravesandestraat 55 oost 1092 aa amsterdam ...,7.7,Hotel Arena,leisure trip couple suite stayed 2 nigh...,netherlands


So now i want to transfer tags from list to string so i can later tokenize them, and while doing it I realize that now it's just a string so i only need to remoove unnescesary symbols

In [120]:
replace_dict = {"'": '', "[": '', "]": '', ",": ''}
for pattern, replacement in replace_dict.items():
    df1['Tags'] = df1['Tags'].str.replace(pattern, replacement)

df1.Tags = df1.Tags.str.lower()


In [127]:
nltk.download('punkt')
nltk.download('stopwords')

def hotel_chosing(country, expectation, top_n=5):
    
    
    df_country = df1[df1['Country'] == country.lower()]
    
    # Tokenize the description
    description_tokens = word_tokenize(expectation)
    
    # Remove stopwords from the description
    description_tokens = [word for word in description_tokens if not word in stopwords.words()]
    
    # Create a mask for rows where the Tags column contains any of the words in the description
    mask = df_country['Tags'].apply(lambda x: any(word in x for word in description_tokens))
    
    # Filter the dataframe using the mask
    df_description = df_country[mask]
    #filtering duplicate columns
    df_description.drop(columns='Tags', inplace=True)
    df_description.drop_duplicates(inplace=True)
    # Sort the dataframe based on the Average_Score and get the top_n hotels
    top_hotels = df_description.sort_values('Average_Score', ascending=False).head(top_n)
    
    return print(top_hotels)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spiri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spiri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [138]:
hotel_chosing('uk', 'busines',10)

                                            Hotel_Address  Average_Score  \
14736   1 suffolk place westminster borough london sw1...            9.6   
185606  41 buckingham palace road westminster borough ...            9.6   
391004  one ham yard westminster borough london w1d 7d...            9.5   
291415  buckingham gate westminster borough london sw1...            9.5   
53782   15 17 charlotte street hotel westminster borou...            9.5   
11266   1 kensington court kensington and chelsea lond...            9.5   
176751  4 richmond mews westminster borough london w1d...            9.5   
14836      1 waterview drive greenwich london se10 0tw uk            9.4   
147508    31 st thomas street southwark london se1 9qu uk            9.4   
38143   12 folgate street city of london london e1 6bx uk            9.4   

                                          Hotel_Name Country  
14736                                Haymarket Hotel      uk  
185606                               

C:\Users\spiri\AppData\Local\Temp\ipykernel_10736\1963807946.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_description.drop(columns='Tags', inplace=True)
C:\Users\spiri\AppData\Local\Temp\ipykernel_10736\1963807946.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_description.drop_duplicates(inplace=True)
